In [55]:
#Dependencies
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import pyproj
import datetime
import folium
from folium import plugins
import random

In [56]:
#Opens up the csv with a random sample of rows (This is done since heatmap breaks above 1 million data points)
file = "parking-citations.csv"
n = sum(1 for line in open(file)) - 1 #number of records in file (excludes header)
s = 500000 #desired sample size
skip = sorted(random.sample(range(1,n+1),n-s)) #sets out which rows to skip to keep the sample
parkingdf = pd.read_csv(file, skiprows=skip)

In [57]:
#Drops all columns not necessary for charts
parkingdf = parkingdf.drop(labels=['Meter Id','Ticket number','Marked Time','Plate Expiry Date','VIN','Route','Agency','Violation code'],axis=1)
parkingdf = parkingdf.replace(to_replace='nan',value=np.nan)
#Drops all null valued rows
parkingdf = parkingdf.dropna()

In [19]:
#Converts date format in the data to a format recognized by python
datetimelist = []
for item in parkingdf['Issue Date']:
    d1 = datetime.datetime.strptime(str(item),"%Y-%m-%dT%H:%M:%S")
    datetimelist.append(d1)
#adds new column with correct date format into Data Frame
parkingdf['Date'] = datetimelist

In [58]:
#Replaces all unusable Lat/Long values with NaN and drops those rows. This becomes new DataFrame
parkingdfheat = parkingdf.replace(to_replace=99999.0,value=np.nan)
parkingdfheat = parkingdfheat.dropna()

In [59]:
#Code to convert Lat/Long in data to decimal format. Also reverses names since data has them backwards
pm = '+proj=lcc +lat_1=34.03333333333333 +lat_2=35.46666666666667 +lat_0=33.5 +lon_0=-118 +x_0=2000000 +y_0=500000.0000000002 +ellps=GRS80 +datum=NAD83 +to_meter=0.3048006096012192 +no_defs'
x1m,y1m = parkingdfheat['Latitude'].values, parkingdfheat['Longitude'].values
x2m,y2m = pyproj.transform(pyproj.Proj(pm,preserve_units = True), pyproj.Proj("+init=epsg:4326"), x1m,y1m)
parkingdfheat['Latitude']=x2m
parkingdfheat['Longitude']=y2m
parkingdfheat = parkingdfheat.rename(columns={'Latitude':'Longitude','Longitude':'Latitude'})

#Creates Heatmap and saves to html object
m = folium.Map([34.0522, -118.2437], zoom_start=11)
citationArr = parkingdfheat[['Latitude', 'Longitude']].as_matrix()
plugins.HeatMap(citationArr, radius=15,max_val=25).add_to(m)
m.save("LAParkingHeatMap(MaxValof25).html")

C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # This is added back by InteractiveShellApp.init_path()


In [38]:
parkingdfheat

,Issue Date,Issue time,RP State Plate,Make,Body Style,Color,Location,Violation Description,Fine amount,Longitude,Latitude
5,2015-12-26T00:00:00,1425.0,CA,HOND,PA,WH,400 WORLD WAY,WHITE CURB,58.0,-118.401448,33.945217
7,2015-12-15T00:00:00,830.0,CA,TOYO,TR,RE,L/L 400 WORLD WAY,WHITE CURB,58.0,-118.401448,33.945217
8,2015-12-26T00:00:00,830.0,CA,INFI,PA,GY,101 WORLD WAY,NO EVIDENCE OF REG,50.0,-118.402021,33.945217
9,2015-12-15T00:00:00,2020.0,CA,TOYO,SU,GY,380 WORLD WAY L/L,WHITE CURB,58.0,-118.401420,33.945217
10,2015-12-15T00:00:00,1935.0,CA,TOYO,PA,SI,600 WORLD WAY U/L,WHITE CURB,58.0,-118.401733,33.945217
11,2015-12-22T00:00:00,1000.0,CA,DODG,TR,WH,500 WORLD WAY,WHITE CURB,58.0,-118.401591,33.945217
12,2015-12-15T00:00:00,1520.0,CA,LROV,PA,BK,301 WORLD WAY,DSPLYPLATE A,25.0,-118.407867,33.944388
18,2015-12-24T00:00:00,1717.0,CA,HOND,PA,BK,10889 WELLWORTH AVE,NO STOP/STAND PM,93.0,-118.442465,34.057059
19,2015-12-24T00:00:00,805.0,CA,ACUR,PA,SI,3521 HUGHES AVE,NO PARK/STREET CLEAN,73.0,-118.402987,34.028757
21,2015-12-24T00:00:00,1219.0,CA,ACUR,PA,GY,3233 SEPULVEDA BLVD,NO PARK/STREET CLEAN,73.0,-118.425901,34.023895
